# Direct diagonalization of tight-binding sd-model with on-site Coulomb repulsion.

In [1]:
#math modules
import math
import numpy as np

import itertools #module for the combinatorial functions

import time

In [5]:
# Kinetic energy
n_site = 2
n_states = math.pow(2, n_site)
# s site system, 2 electrons, NO periodic boundary conditions
from numpy import linalg as LA
U_c = 10
t = -1
ham = [[-U_c, t, t, 0], [t, U_c, 0, t], [t, 0, U_c, t], [0, t, t, -U_c]]
eigenvalues, eigenvectors = LA.eig(np.array(ham))
#eigen-vectors are columns (not rows)
#print(str(eigenvalues))
#print(str(math.sqrt(U_c * U_c + 4 * t * t)))
#print(str(eigenvectors))

In [7]:
# test of intertool libtrary functions
aa=list(itertools.combinations('012345', 3))
#print(str(aa))

In [38]:
# Function converting state representation to spin-up - spin-down representatio
def convert_to_spin(list_of_comb, n_nodes, n_el):
    list_f = []
    for i in range(len(list_of_comb)):
        state = ''
        for j in range(n_el):
            nn = int(list_of_comb[i][j])
            
            if nn >= n_nodes:
                nn_1 = nn - n_nodes
                state = state + str(nn_1)
                state = state + 'd_'
            else:
                state = state + str(nn)
                state = state + 'u_'
        list_f.append(state)
    return list_f
zz = convert_to_spin(aa, 2, 2)
print(zz)

['0u_1u_', '0u_0d_', '0u_1d_', '1u_0d_', '1u_1d_', '0d_1d_']


In [51]:
# Kinetic enerrgy matrix elemenet for the tight-binding model with nearest neighbor jums only
def kinetic_matrix_element(state_1, state_2, n_el, n_nodes, t, periodic):
    jumps = 0
    if periodic == True:
        for i in range(n_el):
            n_1 = int(state_1[i])
            n_2 = int(state_2[i])
            if n_1 == n_2:
                jumps = jumps
            elif n_1 == 0 and n_2 == n_nodes - 1:
                jumps = jumps + 1
            elif n_1 == n_nodes - 1 and n_2 == 0:
                jumps = jumps + 1
            elif n_1 == n_nodes and n_2 == 2 * n_nodes - 1:
                jumps = jumps + 1
            elif n_1 == 2 * n_nodes - 1 and n_2 == n_nodes:
                jumps = jumps + 1    
            elif (n_1 == n_2 + 1 and n_1 != n_nodes) or (n_1 == n_2 - 1 and n_1 != n_nodes - 1):
                jumps = jumps + 1
            else:
                jumps = jumps + 2
        me = 0
        if jumps == 1:
            me = -t
    else:
        for i in range(n_el):
            n_1 = int(state_1[i])
            n_2 = int(state_2[i])
            if n_1 == n_2:
                jumps = jumps
            elif n_1 == 0 and n_2 == n_nodes - 1 and n_nodes > 2:
                jumps = jumps + 2
            elif n_1 == n_nodes - 1 and n_2 == 0 and n_nodes > 2:
                jumps = jumps + 2
            elif n_1 == n_nodes and n_2 == 2 * n_nodes - 1 and n_nodes > 2:
                jumps = jumps + 2
            elif n_1 == 2 * n_nodes - 1 and n_2 == n_nodes and n_nodes > 2:
                jumps = jumps  + 2
            elif (n_1 == n_2 + 1 and n_1 != n_nodes) or (n_1 == n_2 - 1 and n_1 != n_nodes - 1):
                jumps = jumps + 1
            else:
                jumps = jumps + 2
        me = 0
        if jumps == 1:
            me = -t

    return me

#kinetic_matrix_element(aa[0], aa[3], 2, 2, 1)

In [16]:
# Coulomb energy matrix elemenet for on-site Coulomb repulsion
def Coulomb_matrix_element(state_1, state_2, n_el, n_nodes, U_c):
    me = 0
    if state_1 == state_2:
        st = [-1 ] * n_nodes * 2
        for i in range(n_el):
            st[int(state_1[i])] = 1
        #print(st)
        for i in range(n_nodes):
            me = me + U_c / 2 * st[i] * st[i + n_nodes]
    else:
        me = 0
    return me
#Coulomb_matrix_element(aa[1], aa[1], 2, 2, 1)

In [17]:
# sd interaction energy matrix elemenet. Uniform magnetization along z of the localized spins is assumed
def SD_matrix_element(state_1, state_2, n_el, n_nodes, J):
    me = 0
    if state_1 == state_2:
        st = [-1 ] * n_nodes * 2
        for i in range(n_el):
            if int(state_1[i]) < n_nodes:
                me = me - J
            else:
                me = me + J
    return me
#state_number = 4
#SD_matrix_element(aa[state_number], aa[state_number], 3, 3, 1)

In [18]:
# Full matrices describing kinetic energy, Coulomb energy and total energy of the system
def kin_en_matr(states, n_el, n_nodes, t, periodic):
    m_size = len(states)
    k_m = np.zeros((m_size, m_size))
    for i in range(m_size):
        for j in range (m_size):
            k_m[i][j] = kinetic_matrix_element(states[i], states[j], n_el, n_nodes, t, periodic)
    return k_m

def Coul_en_matr(states, n_el, n_nodes, t):
    m_size = len(states)
    k_m = np.zeros((m_size, m_size))
    for i in range(m_size):
        for j in range (m_size):
            k_m[i][j] = Coulomb_matrix_element(states[i], states[j], n_el, n_nodes, t)
    return k_m
    
def SD_en_matr(states, n_el, n_nodes, J):
    m_size = len(states)
    k_m = np.zeros((m_size, m_size))
    for i in range(m_size):
        for j in range (m_size):
            k_m[i][j] = SD_matrix_element(states[i], states[j], n_el, n_nodes, J)
    return k_m

def ham_tot(states, n_el, n_nodes, t, U_c, J, periodic):
    return kin_en_matr(states, n_el, n_nodes, t, periodic) + Coul_en_matr(states, n_el, n_nodes, U_c) + SD_en_matr(states, n_el, n_nodes, J)

In [41]:
#function shown eigenstate of the system in the spin-up - spin_down form
def show_state(states, state):
    state_str = ''
    ll = len(state)
    for i in range(ll):
        if abs(state[i]) >= 0.01:
            ss = convert_to_spin(states, n_nodes, n_el)
            sss = f"{state[i]:.2f}"
            state_str = state_str + sss + ' * ' + str(ss[i]) + ' + '
    return state_str

In [23]:
#set of tasks
tasks = []

#task 1
nodes_number = 2 #number of nodes
electrons_number = 2
J = 0.2 #s-d exchange constant
t = 1 #hopping matrix element (kinetic energy coefficient)
U_c = 10
opt_method = 'cobyla'

tasks.append([nodes_number, electrons_number, J, t, U_c, opt_method])

#task 2
nodes_number = 2 #number of nodes
electrons_number = 2
J = 0.1 #s-d exchange constant
t = 4 #hopping matrix element (kinetic energy coefficient)
U_c = 10
opt_method = 'cobyla'

tasks.append([nodes_number, electrons_number, J, t, U_c, opt_method])

#task 3
nodes_number = 2 #number of nodes
electrons_number = 2
J = 0.2 #s-d exchange constant
t = 1 #hopping matrix element (kinetic energy coefficient)
U_c = 0.1
opt_method = 'cobyla'

tasks.append([nodes_number, electrons_number, J, t, U_c, opt_method])

In [53]:
#Sollving tight-binding model with Coulomb repulsion

for i_task in range(len(tasks)):
    n_nodes = tasks[i_task][0] #number of nodes
    n_el = tasks[i_task][1]
    J = tasks[i_task][2] #s-d exchange constant
    t = tasks[i_task][3] #hopping matrix element (kinetic energy coefficient)
    U_c = tasks[i_task][4]
    periodic = False
    
    ll = ''
    for i in range(n_nodes * 2):
        ll = ll + str(i)
    aa=list(itertools.combinations(ll, n_el))
    ham = ham_tot(aa, n_el, n_nodes, t, U_c, J, periodic)
    eigenvalues, eigenvectors = LA.eig(np.array(ham))
    eig = eigenvectors.T
    #Outpur energies
    print('Task: ' + str(i_task))
    print('All eigenvalue: ' + str(eigenvalues))
    g_en = min(eigenvalues)
    g_state_n = np.argmin(eigenvalues)
    new_eigenvalues = eigenvalues[eigenvalues != g_en] # Excludes '3'
    print(eig)
    ex_en = min(new_eigenvalues)
    print('Ground state: ' + str(g_en))
    print('1st excited state: ' + str(ex_en))
    print('Gap: ' + str(ex_en - g_en))
    
    #Show wave function in the qubits representation
    state_number = g_state_n
    n_states = pow(2, 2 * n_nodes)
    st_int = [0] * len(aa)
    for i_state in range(len(aa)):
        st_int[i_state] = 0
        for i_el in range(n_el):
            st_int[i_state] = st_int[i_state] + pow(2,int(aa[i_state][i_el]))
    full_wf = [0] * pow(2, 2 * n_nodes)
    for i_state in range(len(aa)):
        full_wf[st_int[i_state]] = eig[state_number][i_state]
    full_wf = full_wf[::-1]
    print('Ground state WF in qubits representation: ' + str(full_wf))
    
    #Show wave function in the spin representation
    print(show_state(aa, eig[state_number]))
    print('\n')

Task: 0
All eigenvalue: [-10.19803903  10.          10.19803903 -10.         -10.4
  -9.6       ]
[[ 0.00000000e+00  6.96766179e-02  7.03665523e-01  7.03665523e-01
   6.96766179e-02  0.00000000e+00]
 [ 0.00000000e+00  7.07106781e-01  4.77490711e-16  4.77490711e-16
  -7.07106781e-01  0.00000000e+00]
 [ 0.00000000e+00 -7.03665523e-01  6.96766179e-02  6.96766179e-02
  -7.03665523e-01  0.00000000e+00]
 [ 0.00000000e+00  4.45756037e-17 -7.07106781e-01  7.07106781e-01
  -1.00709083e-16  0.00000000e+00]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]
Ground state: -10.4
1st excited state: -10.19803902718557
Gap: 0.2019609728144296
Ground state WF in qubits representation: [0, 0, 0, 0.0, 0, 0.0, 0.0, 0, 0, 0.0, 0.0, 0, 1.0, 0, 0, 0]
1.00 * 0u_1u_ + 


Task: 1
All eigenvalue: [-12.80624847  10.          12.80624847 -10.         -10.2
  -9.8  